In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.datasets as dsets

In [32]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_size = 784
num_classes = 10
hidden_size = 500
num_epochs = 3
batch_size = 100
learning_rate = 0.001

In [33]:
transform = torchvision.transforms.Compose(
    [torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)
testset= torchvision.datasets.MNIST(root='./data', train= False, transform=transform)

In [34]:
train_loader= torch.utils.data.DataLoader(dataset= trainset, batch_size=batch_size, shuffle= True)
test_loader= torch.utils.data.DataLoader(dataset= testset, batch_size= batch_size, shuffle= False)

In [35]:
class Net(nn.Module):
    def __init__ (self, input_size,hidden_size, num_classes):
        super(Net,self).__init__()
        self.fc1= nn.Linear(input_size, hidden_size)
        self.relu= nn.ReLU()
        self.fc2= nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        out= self.fc1(x)
        out= self.relu(out)
        out= self.fc2(out)
        return out
    
model= Net(input_size, hidden_size, num_classes).to(device)

In [36]:
criterion= nn.CrossEntropyLoss()
optimizer= torch.optim.Adam(model.parameters(), lr= learning_rate)

In [37]:
total_steps= len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images= images.reshape(-1, 28*28)
        labels= labels.to(device)
        
        output= model(images)
        loss= criterion(output, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_steps, loss.item()))

Epoch [1/3], Step [100/600], Loss: 0.5838
Epoch [1/3], Step [200/600], Loss: 0.4552
Epoch [1/3], Step [300/600], Loss: 0.3216
Epoch [1/3], Step [400/600], Loss: 0.2512
Epoch [1/3], Step [500/600], Loss: 0.1445
Epoch [1/3], Step [600/600], Loss: 0.2510
Epoch [2/3], Step [100/600], Loss: 0.2204
Epoch [2/3], Step [200/600], Loss: 0.2900
Epoch [2/3], Step [300/600], Loss: 0.1339
Epoch [2/3], Step [400/600], Loss: 0.1509
Epoch [2/3], Step [500/600], Loss: 0.1318
Epoch [2/3], Step [600/600], Loss: 0.0877
Epoch [3/3], Step [100/600], Loss: 0.0729
Epoch [3/3], Step [200/600], Loss: 0.1163
Epoch [3/3], Step [300/600], Loss: 0.1408
Epoch [3/3], Step [400/600], Loss: 0.1137
Epoch [3/3], Step [500/600], Loss: 0.0896
Epoch [3/3], Step [600/600], Loss: 0.1250


In [38]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))


Accuracy of the network on the 10000 test images: 96.72 %
